In [1]:
import fastjet as fj
import time
import pyhepmc

In [7]:
def makeJets(data, R):
    jet_def = fj.JetDefinition(fj.cambridge_aachen_algorithm, R)
    jet_cluster = fj.ClusterSequence(data, jet_def)

    return jet_cluster

In [3]:
def read_hepmc_file(FILE):
    total = []
    with pyhepmc.open(f"./data/{FILE}", "r") as f:
        
        for i, event in enumerate(f):
            particles = []
            for j, particle in enumerate(event.particles):
                if particle.status == 1: 
                # Access particle properties such as momentum, energy, etc.
                # print(particle.momentum.px)
                    px, py, pz = particle.momentum.px, particle.momentum.py, particle.momentum.pz
                    energy = particle.momentum.e

                    particles.append(fj.PseudoJet(px, py, pz, energy))
            total.append(particles)

    return total

In [4]:
fileName = ["events-ee-Z.hepmc3", "events-ee-H.hepmc3", \
                "events-pp-0.5TeV-5GeV.hepmc3", "events-pp-1TeV-5GeV.hepmc3",\
                "events-pp-2TeV-5GeV.hepmc3", "events-pp-1TeV.hepmc3", "events-pp-2TeV.hepmc3",\
                "events-pp-5TeV-10GeV.hepmc3", "events-pp-8TeV-20GeV.hepmc3",\
                "events-pp-13TeV-20GeV.hepmc", "events-pp-20TeV-20GeV.hepmc3", "events-pp-20TeV-50GeV.hepmc3",\
                "events-pp-30TeV-50GeV.hepmc3"]

In [93]:
trials = 16

with open("./results/time_python_plain.csv", "w") as FILE:
    FILE.write("FileName,mean_particles,n_samples,time_per_event\n")
    for eventFILE in fileName:
        allEvents = read_hepmc_file(eventFILE)

        avgTime = 0.0
        for j in range(trials):
            
            count = 0
            start = time.time()
            for i in range(100):
                count += len(allEvents[i])
                cluster = makeJets(allEvents[i], 0.4, fj.N2Plain)
            
            end = time.time() - start
            avgTime += end

            # count += len(particle_data)
        avgTime /= trials

        print(f"{eventFILE}\t{count/100}\t{trials}\t{avgTime*10**4}\n")
        FILE.write(f"{eventFILE},{count/100},{trials},{avgTime*10**4}\n")


events-ee-Z.hepmc3	43.05	16	66.29317998886108

events-ee-H.hepmc3	64.97	16	107.51619935035706

events-pp-0.5TeV-5GeV.hepmc3	112.62	16	555.4723739624023

events-pp-1TeV-5GeV.hepmc3	160.36	16	1118.127554655075

events-pp-2TeV-5GeV.hepmc3	188.21	16	1537.9343926906586

events-pp-1TeV.hepmc3	226.98	16	1639.4996643066406

events-pp-2TeV.hepmc3	226.98	16	1643.0480778217316

events-pp-5TeV-10GeV.hepmc3	284.15	16	2642.2417163848877

events-pp-8TeV-20GeV.hepmc3	354.18	16	3621.6507852077484

events-pp-13TeV-20GeV.hepmc	431.18	16	5136.139839887619

events-pp-20TeV-20GeV.hepmc3	524.59	16	6977.768391370773

events-pp-20TeV-50GeV.hepmc3	553.64	16	7087.072134017944

events-pp-30TeV-50GeV.hepmc3	632.29	16	8945.840299129486



In [10]:
trials = 16
with open("./results/time-python-substructure.csv", "w") as FILE:
    FILE.write("FileName,mean_particles,n_samples,filter_time,trimming_time,massdrop_time\n")
    for eventFILE in fileName:
        allEvents = read_hepmc_file(eventFILE)

        avgTime = 0.0
        avg_filter_time = 0.0
        avg_trimming_time = 0.0
        avg_massdrop_time = 0.0
        
        for j in range(trials):

            count = 0
            
            for i in range(100):
                count += len(allEvents[i])
                cluster = makeJets(allEvents[i], 1.0)
                jets = cluster.inclusive_jets(2.0)
                # mu = 0.67
                # ycut = 0.09
                mass_drop_tagger = fj.MassDropTagger(0.67, 0.09)
                start = time.time()
                final_jets = [mass_drop_tagger(jet) for jet in jets]
                end = time.time()
                avg_massdrop_time += end - start

                filtering = fj.Filter(fj.JetDefinition(fj.cambridge_algorithm, 0.3), fj.SelectorNHardest(3))
                start = time.time()
                final_jets = [filtering(jet) for jet in jets]
                end = time.time()
                avg_filter_time += end - start

                trimming = fj.Filter(fj.JetDefinition(fj.cambridge_algorithm, 0.3), fj.SelectorPtFractionMin(0.3))
                start = time.time()
                final_jets = [trimming(jet) for jet in jets]
                end = time.time()
                avg_trimming_time += end - start

                # JH_tagger = fj.JHTopTagger(0.1, 0.19, 0.7, 80.4)
                # start = time.time()
                # final_jets = [JH_tagger(jet) for jet in jets]                
                # end = time.time() - start


        avg_massdrop_time /= trials
        avg_filter_time /= trials
        avg_trimming_time /= trials

        print(f"{eventFILE}\t{count/100}\t{trials}\t{avg_filter_time*10**4}\t{avg_trimming_time*10**4}\t{avg_massdrop_time*10**4}\n")
        FILE.write(f"{eventFILE},{count/100},{trials},{avg_filter_time*10**4},{avg_trimming_time*10**4},{avg_massdrop_time*10**4}\n")

events-ee-Z.hepmc3	43.05	16	14.56528902053833	13.60669732093811	5.011856555938721

events-ee-H.hepmc3	64.97	16	17.2482430934906	15.836209058761597	5.757510662078857

events-pp-0.5TeV-5GeV.hepmc3	112.62	16	38.07276487350464	35.53956747055054	10.997653007507324

events-pp-1TeV-5GeV.hepmc3	160.36	16	55.45705556869507	52.48740315437317	15.67155122756958

events-pp-2TeV-5GeV.hepmc3	188.21	16	67.606121301651	63.40056657791138	18.741637468338013

events-pp-1TeV.hepmc3	226.98	16	75.08590817451477	70.65460085868835	20.175427198410034

events-pp-2TeV.hepmc3	226.98	16	76.04971528053284	71.44778966903687	20.581036806106567

events-pp-5TeV-10GeV.hepmc3	284.15	16	106.12621903419495	99.56702589988708	28.071850538253784

events-pp-8TeV-20GeV.hepmc3	354.18	16	117.90469288825989	110.88907718658447	30.665993690490723

events-pp-13TeV-20GeV.hepmc	431.18	16	146.39884233474731	139.13512229919434	41.59405827522278

events-pp-20TeV-20GeV.hepmc3	524.59	16	238.80288004875183	197.11166620254517	53.66936326026916